In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

In [2]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)
print("done!")

done!


In [110]:

# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)

In [4]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    
def get_model():
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 1), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16,3,3, activation=p_activation) (input_1)
    img_1 = Conv2D(16,3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
    img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
    img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(128, 3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, 3,3, activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2]))
    
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model(input_1,  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
model = get_model()
model.summary()

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
X_1 (InputLayer)                 (None, 75, 75, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 73, 73, 16)    160         X_1[0][0]                        
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 71, 71, 16)    2320        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 35, 35, 16)    0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [114]:
import utils

batch_size=32

gen = ImageDataGenerator(vertical_flip = True)

#gen = ImageDataGenerator(zoom_range = 0.1)


#gen = ImageDataGenerator(horizontal_flip = True,
#                         vertical_flip = True,
#                         width_shift_range = 0.1,
#                         height_shift_range = 0.1,
#                         zoom_range = 0.1,
#                         rotation_range = 40)


# Finally create out generator
#gen_flow = utils.gen_flow_for_two_inputs(X_train, X_angle_train, y_train, gen)

genX1 = gen.flow(X_train,y_train,  batch_size=32,seed=666)

In [119]:
file_path = "/mnt/extDisk/courses/data/statoil/data/data_aug/.model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=30)

model = get_model()
model.fit_generator(genX1, validation_data=(X_valid, y_valid),
                    steps_per_epoch=len(X_train) / 32, epochs=40, callbacks=callbacks)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 

Epoch 1/40
18/18 [==============================] - 11s - loss: 0.9261 - acc: 0.5253 - val_loss: 0.6403 - val_acc: 0.5850
Epoch 2/40
18/18 [==============================] - 0s - loss: 0.9095 - acc: 0.5635 - val_loss: 0.6472 - val_acc: 0.5850
Epoch 3/40
18/18 [==============================] - 0s - loss: 0.8086 - acc: 0.6046 - val_loss: 0.7286 - val_acc: 0.5900
Epoch 4/40
18/18 [==============================] - 0s - loss: 0.8157 - acc: 0.5687 - val_loss: 0.7006 - val_acc: 0.6300
Epoch 5/40
18/18 [==============================] - 0s - loss: 0.7905 - acc: 0.5926 - val_loss: 0.6462 - val_acc: 0.6300
Epoch 6/40
18/18 [==============================] - 0s - loss: 0.7624 - acc: 0.6073 - val_loss: 0.8684 - val_acc: 0.6650
Epoch 7/40
18/18 [==============================] - 0s - loss: 0.7742 - acc: 0.5625 - val_loss: 0.7176 - val_acc: 0.6150
Epoch 8/40
18/18 [==============================] - 0s - loss: 0.7388 - acc: 0.5980 - val_loss: 0.7937 - val_acc: 0.6650
Epoch 9/40
18/18 [=============

In [120]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate(X_train, y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate(X_valid, y_valid, verbose=1, batch_size=200))

Train evaluate:
600/602 [============================>.] - ETA: 0s[0.24802597656078712, 0.88870432606567185]
####################
watch list evaluate:
200/200 [==============================] - 0s
[0.28891438245773315, 0.84499996900558472]


In [121]:
prediction = model.predict(X_test, verbose=1, batch_size=200)

8200/8424 [============================>.] - ETA: 0s

In [122]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.008860
1,4023181e,0.082705
2,b20200e4,0.543820
3,e7f018bb,0.998968
4,4371c8c3,0.107391
5,a8d9b1fd,0.999719
6,29e7727e,0.105111
7,92a51ffb,0.981857
8,c769ac97,0.014250
9,aee0547d,0.003939


In [123]:
submission.to_csv("/mnt/extDisk/courses/data/statoil/data/data_aug/data_augmentation_band1.csv", index=False)

In [109]:
X_train.shape

(602, 75, 75, 1)